# Part 2: 수작업 Knowledge Graph 구축

**소요 시간**: 약 2시간  
**난이도**: 초급  
**목표**: 온톨로지를 직접 설계하고, 뉴스 기사에서 수동으로 엔티티/관계를 추출하여 Neo4j에 적재합니다.

---

## 학습 순서

1. 환경 설정 및 데이터 로드
2. 온톨로지 설계 워크숍
3. 수동 엔티티 추출
4. Meta-Dictionary 생성
5. Neo4j 적재
6. 시각화 + 검증 쿼리
7. 연습 문제

---

## Part 1 복습

Part 1에서 우리는 7개 노드로 된 제조 그래프를 만들고,  
Multi-hop 쿼리가 왜 강력한지 체험했습니다.

이번 Part 2에서는 한 단계 더 나아가서:  
**비정형 텍스트(뉴스 기사)에서 지식 그래프를 수작업으로 만드는 전체 프로세스**를 배웁니다.

> **왜 수작업부터?** Part 3에서 LLM으로 자동화하기 전에,  
> 사람이 직접 해봐야 LLM 결과의 품질을 판단할 수 있습니다.

---
## 1. 환경 설정

### 1.1 패키지 및 Neo4j 연결

In [ ]:
import os
import json
from dotenv import load_dotenv
from neo4j import GraphDatabase

# 환경 변수 로드
load_dotenv()
load_dotenv(dotenv_path="../.env")

NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "")

# Neo4j 연결
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

try:
    driver.verify_connectivity()
    print("[OK] Neo4j 연결 성공!")
except Exception as e:
    print(f"[ERROR] 연결 실패: {e}")

# 편의 함수
def run_query(query, parameters=None, print_result=True):
    """Cypher 쿼리를 실행하고 결과를 반환합니다."""
    with driver.session() as session:
        result = session.run(query, parameters or {})
        records = [record.data() for record in result]
        if print_result:
            if records:
                for i, rec in enumerate(records, 1):
                    print(f"  [{i}] {rec}")
            else:
                print("  (결과 없음)")
        return records

print("[OK] 환경 설정 완료")

### 1.2 샘플 뉴스 기사 데이터 로드

한국 IT/반도체 기업 관련 뉴스 기사 10개를 사용합니다.  
`data/news_articles.json` 파일이 없으면 아래에서 직접 생성합니다.

In [ ]:
# 샘플 뉴스 기사 데이터 (10개)
# 실제 뉴스를 참고하여 구성한 학습용 가상 데이터입니다.

sample_articles = [
    {
        "id": "article_01",
        "title": "삼성전자, 2나노 GAA 공정 양산 본격화",
        "content": "삼성전자가 세계 최초로 2나노 GAA(Gate-All-Around) 공정 양산에 돌입했다. "
                   "이재용 회장은 평택 캠퍼스를 방문해 반도체 사업 전략을 점검했다. "
                   "삼성전자는 갤럭시 S25에 자체 개발한 Exynos 2500 칩을 탑재할 계획이며, "
                   "이를 위해 2나노 공정 수율 개선에 집중하고 있다.",
        "date": "2024-11-15",
        "source": "전자신문"
    },
    {
        "id": "article_02",
        "title": "SK하이닉스, HBM4 개발 완료... NVIDIA 납품 확대",
        "content": "SK하이닉스가 차세대 고대역폭 메모리 HBM4 개발을 완료하고, "
                   "NVIDIA의 차세대 GPU인 Blackwell에 공급하기로 했다. "
                   "SK하이닉스 곽노정 CEO는 AI 반도체 시장에서의 기술 리더십을 강조했다. "
                   "SK하이닉스는 이천 공장에서 HBM4를 생산할 예정이다.",
        "date": "2024-11-20",
        "source": "조선비즈"
    },
    {
        "id": "article_03",
        "title": "국민연금, 삼성전자 지분 추가 매입",
        "content": "국민연금이 삼성전자 지분을 2조원 규모로 추가 매입했다. "
                   "국민연금은 이미 삼성전자 최대 기관투자자로, "
                   "이번 매입으로 지분율이 11.2%까지 상승했다. "
                   "국민연금은 SK하이닉스에도 1.5조원을 투자하고 있다.",
        "date": "2024-12-01",
        "source": "한국경제"
    },
    {
        "id": "article_04",
        "title": "네이버, 자체 AI 칩 개발 추진",
        "content": "네이버가 TSMC와 협력하여 자체 AI 추론 전용 칩을 개발한다고 밝혔다. "
                   "네이버 최수연 CEO는 'HyperCLOVA X의 추론 비용을 50% 절감하겠다'고 말했다. "
                   "네이버는 이를 위해 TSMC의 3나노 공정을 활용할 예정이며, "
                   "자회사인 네이버클라우드에서 서비스를 운영할 계획이다.",
        "date": "2024-12-05",
        "source": "매일경제"
    },
    {
        "id": "article_05",
        "title": "카카오-삼성전자, AI 스마트홈 협력",
        "content": "카카오와 삼성전자가 AI 기반 스마트홈 플랫폼 구축을 위한 전략적 파트너십을 맺었다. "
                   "카카오의 AI 비서 '카나나'를 삼성전자의 가전제품에 탑재하는 것이 핵심이다. "
                   "카카오 정신아 CEO와 삼성전자 한종희 부회장이 MOU에 서명했다. "
                   "삼성전자는 비스포크 AI 냉장고를 첫 적용 제품으로 선정했다.",
        "date": "2024-12-10",
        "source": "중앙일보"
    },
    {
        "id": "article_06",
        "title": "블랙록, 한국 반도체 투자 확대",
        "content": "글로벌 투자기관 블랙록이 한국 반도체 기업에 대한 투자를 확대한다. "
                   "블랙록은 삼성전자에 5조원, SK하이닉스에 3조원 규모의 지분을 보유하고 있다. "
                   "블랙록의 래리 핑크 CEO는 'AI 시대에 한국 반도체는 핵심 인프라'라고 평가했다.",
        "date": "2024-12-12",
        "source": "파이낸셜뉴스"
    },
    {
        "id": "article_07",
        "title": "삼성전자 vs SK하이닉스, HBM 시장 경쟁 심화",
        "content": "삼성전자와 SK하이닉스의 HBM 시장 경쟁이 치열해지고 있다. "
                   "삼성전자는 HBM3E 12단 제품으로 NVIDIA 인증을 추진 중이며, "
                   "SK하이닉스는 이미 HBM3E를 NVIDIA에 대량 납품하고 있다. "
                   "두 회사 모두 용인과 이천에 신규 팹을 건설 중이다.",
        "date": "2024-12-15",
        "source": "디지털타임스"
    },
    {
        "id": "article_08",
        "title": "TSMC, 삼성전자 파운드리 고객 유치 경쟁",
        "content": "TSMC가 삼성전자의 파운드리 고객을 적극 유치하고 있다. "
                   "퀄컴이 차세대 스냅드래곤 칩을 TSMC 3나노 공정으로 전환한 것이 대표적이다. "
                   "삼성전자는 이에 대응해 2나노 GAA 공정의 수율을 높이고 있으며, "
                   "구글과 테슬라의 차세대 칩 수주를 추진하고 있다.",
        "date": "2024-12-18",
        "source": "블룸버그코리아"
    },
    {
        "id": "article_09",
        "title": "삼성SDI, 전고체 배터리 양산 추진",
        "content": "삼성전자 계열사인 삼성SDI가 전고체 배터리 양산을 2027년으로 앞당긴다. "
                   "삼성SDI는 천안 공장에서 파일럿 라인을 가동 중이며, "
                   "BMW와 전고체 배터리 공급 계약을 체결했다. "
                   "삼성SDI 최윤호 CEO는 '전기차 시장의 게임 체인저가 될 것'이라고 했다.",
        "date": "2024-12-20",
        "source": "서울경제"
    },
    {
        "id": "article_10",
        "title": "공정거래위원회, 반도체 업계 공정 거래 점검",
        "content": "공정거래위원회가 반도체 업계의 하도급 거래 실태를 점검한다. "
                   "삼성전자와 SK하이닉스의 협력업체 대금 지급 현황이 조사 대상이다. "
                   "공정위는 특히 소재/부품/장비 분야 중소기업의 거래 공정성에 주목하고 있다.",
        "date": "2024-12-22",
        "source": "연합뉴스"
    }
]

# data/ 디렉토리에 저장
os.makedirs("data", exist_ok=True)
with open("data/news_articles.json", "w", encoding="utf-8") as f:
    json.dump(sample_articles, f, ensure_ascii=False, indent=2)

print(f"[OK] {len(sample_articles)}개 뉴스 기사 로드 완료")
print(f"\n기사 목록:")
for i, article in enumerate(sample_articles, 1):
    print(f"  [{i:2d}] {article['title']} ({article['date']})")

In [ ]:
# data/news_articles.json 파일에서 로드 (이미 파일이 있는 경우)
with open("data/news_articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

print(f"[OK] {len(articles)}개 기사 로드 완료")

---
## 2. 온톨로지 설계 워크숍

Knowledge Graph를 만들려면 먼저 **온톨로지(Ontology)**를 설계해야 합니다.  
온톨로지란 '우리 도메인에서 어떤 종류의 엔티티와 관계가 존재하는가'를 정의한 스키마입니다.

### 설계 원칙

1. **친숙한 도메인을 선택하라** - 관계의 정확성을 직관적으로 판단할 수 있어야 합니다.
2. **엔티티 타입은 5~7개로 시작하라** - 너무 많으면 복잡해지고, 너무 적으면 표현력이 부족합니다.
3. **관계 타입은 10개 이하로** - 처음부터 모든 관계를 정의하지 마세요. 필요할 때 추가하세요.
4. **방향이 중요하다** - `A-[INVESTS_IN]->B`와 `B-[INVESTS_IN]->A`는 완전히 다른 의미입니다.

### 2.1 엔티티 타입 정의 (5개)

우리 도메인 "한국 IT/반도체 기업"에서 핵심 엔티티 타입 5가지를 정의합니다.

| Entity Type | 설명 | 예시 |
|-------------|------|------|
| **Company** | 기업 | 삼성전자, SK하이닉스, 네이버, 카카오, TSMC |
| **Person** | 인물 (경영진) | 이재용, 곽노정, 최수연, 래리 핑크 |
| **Product** | 제품/기술 | 갤럭시 S25, HBM4, Exynos 2500, HyperCLOVA X |
| **Technology** | 핵심 기술 | 2나노 GAA, HBM, 전고체 배터리 |
| **Investment** | 투자기관 | 국민연금, 블랙록 |

In [ ]:
# 온톨로지를 Python 딕셔너리로 정의

ontology = {
    "entity_types": {
        "Company": {
            "description": "기업 (IT, 반도체, 자동차 등)",
            "properties": ["name", "industry", "headquarters"],
            "examples": ["삼성전자", "SK하이닉스", "네이버", "카카오", "TSMC", "NVIDIA"]
        },
        "Person": {
            "description": "인물 (CEO, 회장 등 경영진)",
            "properties": ["name", "title", "affiliation"],
            "examples": ["이재용", "곽노정", "최수연", "래리 핑크"]
        },
        "Product": {
            "description": "제품 (하드웨어, 소프트웨어, 서비스)",
            "properties": ["name", "category", "release_year"],
            "examples": ["갤럭시 S25", "HBM4", "Exynos 2500", "HyperCLOVA X", "카나나"]
        },
        "Technology": {
            "description": "핵심 기술 (공정, 아키텍처, 소재)",
            "properties": ["name", "domain", "maturity"],
            "examples": ["2나노 GAA", "HBM", "3나노 공정", "전고체 배터리"]
        },
        "Investment": {
            "description": "투자기관 (연기금, 자산운용사)",
            "properties": ["name", "type", "aum"],
            "examples": ["국민연금", "블랙록"]
        }
    }
}

print("=== 엔티티 타입 정의 ===")
for etype, info in ontology["entity_types"].items():
    print(f"\n  [{etype}] {info['description']}")
    print(f"    속성: {', '.join(info['properties'])}")
    print(f"    예시: {', '.join(info['examples'][:3])}")

### 2.2 관계 타입 정의 (9개)

엔티티들 사이에 어떤 관계가 존재하는지 정의합니다.

| 관계 | 방향 | 의미 | 예시 |
|------|------|------|------|
| FOUNDED_BY | Company -> Person | 설립/경영 | 삼성전자 -> 이재용 |
| DEVELOPS | Company -> Product | 개발/생산 | 삼성전자 -> 갤럭시 S25 |
| INVESTS_IN | Investment -> Company | 투자 | 국민연금 -> 삼성전자 |
| ACQUIRES | Company -> Company | 인수 | - |
| PARTNERS_WITH | Company -> Company | 협력/제휴 | 카카오 -> 삼성전자 |
| COMPETES_WITH | Company -> Company | 경쟁 | 삼성전자 -> SK하이닉스 |
| EMPLOYS | Company -> Person | 고용/소속 | SK하이닉스 -> 곽노정 |
| USES_TECH | Company/Product -> Technology | 기술 활용 | 삼성전자 -> 2나노 GAA |
| PRODUCES | Company -> Product | 제품 생산 | SK하이닉스 -> HBM4 |

In [ ]:
# 관계 타입 정의 추가

ontology["relation_types"] = {
    "FOUNDED_BY": {
        "description": "설립/경영 관계",
        "source_types": ["Company"],
        "target_types": ["Person"],
        "properties": ["role"],
        "example": "(삼성전자)-[:FOUNDED_BY {role: '회장'}]->(이재용)"
    },
    "DEVELOPS": {
        "description": "제품/기술 개발 관계",
        "source_types": ["Company"],
        "target_types": ["Product"],
        "properties": ["status", "year"],
        "example": "(삼성전자)-[:DEVELOPS]->(Exynos 2500)"
    },
    "INVESTS_IN": {
        "description": "투자 관계",
        "source_types": ["Investment"],
        "target_types": ["Company"],
        "properties": ["amount", "share_pct"],
        "example": "(국민연금)-[:INVESTS_IN {amount: '2조원', share_pct: 11.2}]->(삼성전자)"
    },
    "ACQUIRES": {
        "description": "인수 관계",
        "source_types": ["Company"],
        "target_types": ["Company"],
        "properties": ["amount", "year"],
        "example": "(A)-[:ACQUIRES]->(B)"
    },
    "PARTNERS_WITH": {
        "description": "전략적 파트너십/협력",
        "source_types": ["Company"],
        "target_types": ["Company"],
        "properties": ["domain", "year"],
        "example": "(카카오)-[:PARTNERS_WITH {domain: 'AI 스마트홈'}]->(삼성전자)"
    },
    "COMPETES_WITH": {
        "description": "경쟁 관계",
        "source_types": ["Company"],
        "target_types": ["Company"],
        "properties": ["market"],
        "example": "(삼성전자)-[:COMPETES_WITH {market: 'HBM'}]->(SK하이닉스)"
    },
    "EMPLOYS": {
        "description": "고용/소속 관계",
        "source_types": ["Company"],
        "target_types": ["Person"],
        "properties": ["title", "since"],
        "example": "(SK하이닉스)-[:EMPLOYS {title: 'CEO'}]->(곽노정)"
    },
    "USES_TECH": {
        "description": "기술 활용 관계",
        "source_types": ["Company", "Product"],
        "target_types": ["Technology"],
        "properties": ["purpose"],
        "example": "(삼성전자)-[:USES_TECH]->(2나노 GAA)"
    },
    "PRODUCES": {
        "description": "제품 생산/출시 관계",
        "source_types": ["Company"],
        "target_types": ["Product"],
        "properties": ["factory", "year"],
        "example": "(SK하이닉스)-[:PRODUCES {factory: '이천'}]->(HBM4)"
    }
}

print("=== 관계 타입 정의 (9개) ===")
for rtype, info in ontology["relation_types"].items():
    src = '/'.join(info['source_types'])
    tgt = '/'.join(info['target_types'])
    print(f"  {src} -[:{rtype}]-> {tgt}  |  {info['description']}")

# 온톨로지 저장
with open("data/ontology.json", "w", encoding="utf-8") as f:
    json.dump(ontology, f, ensure_ascii=False, indent=2)
print(f"\n[OK] 온톨로지를 data/ontology.json에 저장했습니다.")

---
## 3. 수동 엔티티 추출

이제 실제 뉴스 기사에서 엔티티와 관계를 추출해봅시다.  
Part 3에서 LLM으로 자동화하기 전에, **사람이 직접 해보는 것**이 중요합니다.

### 추출 프로세스

1. 기사를 읽는다
2. 엔티티(명사)를 식별한다 -> 타입을 분류한다
3. 관계(동사/서술어)를 식별한다 -> 타입을 분류한다
4. 추출 결과를 구조화된 JSON으로 정리한다

### 추출 템플릿

```python
extraction = {
    "article_id": "article_01",
    "entities": [
        {"name": "삼성전자", "type": "Company", "properties": {"industry": "반도체"}}
    ],
    "relations": [
        {"source": "삼성전자", "target": "Exynos 2500", "type": "DEVELOPS", "properties": {}}
    ]
}
```

In [ ]:
# 기사 1번 내용 확인
article = articles[0]
print(f"=== 기사 #{article['id']} ===")
print(f"제목: {article['title']}")
print(f"날짜: {article['date']} | 출처: {article['source']}")
print(f"\n본문:")
print(f"  {article['content']}")
print("\n" + "=" * 60)
print("\n이 기사에서 어떤 엔티티와 관계를 추출할 수 있을까요?")
print("아래에서 추출 과정을 시연합니다.")

In [ ]:
# === 기사 1번 수동 추출 시연 ===

extraction_01 = {
    "article_id": "article_01",
    "entities": [
        # Company
        {"name": "삼성전자", "type": "Company", "properties": {"industry": "반도체/전자", "headquarters": "수원"}},
        # Person
        {"name": "이재용", "type": "Person", "properties": {"title": "회장", "affiliation": "삼성전자"}},
        # Product
        {"name": "갤럭시 S25", "type": "Product", "properties": {"category": "스마트폰"}},
        {"name": "Exynos 2500", "type": "Product", "properties": {"category": "AP 칩"}},
        # Technology
        {"name": "2나노 GAA", "type": "Technology", "properties": {"domain": "반도체 공정"}}
    ],
    "relations": [
        # 삼성전자 -[EMPLOYS]-> 이재용
        {"source": "삼성전자", "target": "이재용", "type": "EMPLOYS",
         "properties": {"title": "회장"}},
        # 삼성전자 -[DEVELOPS]-> Exynos 2500
        {"source": "삼성전자", "target": "Exynos 2500", "type": "DEVELOPS",
         "properties": {"status": "개발 중"}},
        # 삼성전자 -[DEVELOPS]-> 갤럭시 S25
        {"source": "삼성전자", "target": "갤럭시 S25", "type": "DEVELOPS",
         "properties": {"status": "계획"}},
        # 삼성전자 -[USES_TECH]-> 2나노 GAA
        {"source": "삼성전자", "target": "2나노 GAA", "type": "USES_TECH",
         "properties": {"purpose": "양산"}},
        # 갤럭시 S25 -[USES_TECH]-> Exynos 2500 (제품이 칩을 사용)
        {"source": "갤럭시 S25", "target": "Exynos 2500", "type": "USES_TECH",
         "properties": {"purpose": "AP 탑재"}}
    ]
}

print("=== 기사 1번 추출 결과 ===")
print(f"\n엔티티 {len(extraction_01['entities'])}개:")
for e in extraction_01["entities"]:
    print(f"  [{e['type']:12s}] {e['name']}")

print(f"\n관계 {len(extraction_01['relations'])}개:")
for r in extraction_01["relations"]:
    print(f"  {r['source']} -[:{r['type']}]-> {r['target']}")

In [ ]:
# === 기사 2번 수동 추출 ===

print(f"\n=== 기사 #{articles[1]['id']} ===")
print(f"제목: {articles[1]['title']}")
print(f"본문: {articles[1]['content']}")
print()

In [ ]:
extraction_02 = {
    "article_id": "article_02",
    "entities": [
        {"name": "SK하이닉스", "type": "Company", "properties": {"industry": "반도체", "headquarters": "이천"}},
        {"name": "NVIDIA", "type": "Company", "properties": {"industry": "GPU/AI", "headquarters": "산타클라라"}},
        {"name": "곽노정", "type": "Person", "properties": {"title": "CEO", "affiliation": "SK하이닉스"}},
        {"name": "HBM4", "type": "Product", "properties": {"category": "메모리"}},
        {"name": "Blackwell", "type": "Product", "properties": {"category": "GPU"}},
        {"name": "HBM", "type": "Technology", "properties": {"domain": "고대역폭 메모리"}}
    ],
    "relations": [
        {"source": "SK하이닉스", "target": "곽노정", "type": "EMPLOYS",
         "properties": {"title": "CEO"}},
        {"source": "SK하이닉스", "target": "HBM4", "type": "PRODUCES",
         "properties": {"factory": "이천"}},
        {"source": "SK하이닉스", "target": "NVIDIA", "type": "PARTNERS_WITH",
         "properties": {"domain": "HBM 공급"}},
        {"source": "SK하이닉스", "target": "HBM", "type": "USES_TECH",
         "properties": {"purpose": "메모리 생산"}},
        {"source": "NVIDIA", "target": "Blackwell", "type": "DEVELOPS",
         "properties": {"status": "차세대"}}
    ]
}

print("=== 기사 2번 추출 결과 ===")
print(f"\n엔티티 {len(extraction_02['entities'])}개:")
for e in extraction_02["entities"]:
    print(f"  [{e['type']:12s}] {e['name']}")
print(f"\n관계 {len(extraction_02['relations'])}개:")
for r in extraction_02["relations"]:
    print(f"  {r['source']} -[:{r['type']}]-> {r['target']}")

In [ ]:
# === 기사 3번 수동 추출 ===

print(f"\n=== 기사 #{articles[2]['id']} ===")
print(f"제목: {articles[2]['title']}")
print(f"본문: {articles[2]['content']}")
print()

In [ ]:
extraction_03 = {
    "article_id": "article_03",
    "entities": [
        {"name": "국민연금", "type": "Investment", "properties": {"type": "연기금"}},
        {"name": "삼성전자", "type": "Company", "properties": {"industry": "반도체/전자"}},
        {"name": "SK하이닉스", "type": "Company", "properties": {"industry": "반도체"}}
    ],
    "relations": [
        {"source": "국민연금", "target": "삼성전자", "type": "INVESTS_IN",
         "properties": {"amount": "2조원", "share_pct": 11.2}},
        {"source": "국민연금", "target": "SK하이닉스", "type": "INVESTS_IN",
         "properties": {"amount": "1.5조원"}}
    ]
}

print("=== 기사 3번 추출 결과 ===")
print(f"\n엔티티 {len(extraction_03['entities'])}개:")
for e in extraction_03["entities"]:
    print(f"  [{e['type']:12s}] {e['name']}")
print(f"\n관계 {len(extraction_03['relations'])}개:")
for r in extraction_03["relations"]:
    props = f" ({r['properties']})" if r['properties'] else ""
    print(f"  {r['source']} -[:{r['type']}]-> {r['target']}{props}")

In [ ]:
# 3개 기사 추출 결과 통합
all_extractions = [extraction_01, extraction_02, extraction_03]

# 추출 결과 저장
with open("data/extractions.json", "w", encoding="utf-8") as f:
    json.dump(all_extractions, f, ensure_ascii=False, indent=2)

# 통계
total_entities = sum(len(ex["entities"]) for ex in all_extractions)
total_relations = sum(len(ex["relations"]) for ex in all_extractions)
unique_entity_names = set()
for ex in all_extractions:
    for e in ex["entities"]:
        unique_entity_names.add(e["name"])

print("=== 추출 통계 (기사 3개) ===")
print(f"  총 엔티티 멘션: {total_entities}개")
print(f"  고유 엔티티: {len(unique_entity_names)}개")
print(f"  총 관계: {total_relations}개")
print(f"\n  고유 엔티티 목록: {sorted(unique_entity_names)}")

---
## 4. Meta-Dictionary 생성

**Meta-Dictionary**는 관계 타입별 키워드를 매핑한 사전입니다.  
뉴스 기사에서 특정 키워드가 나오면 어떤 관계 타입에 해당하는지 빠르게 판단할 수 있습니다.

### 왜 필요한가?

- **수동 추출 시**: 사람이 일관된 기준으로 관계를 분류할 수 있음
- **LLM 자동 추출 시 (Part 3)**: 프롬프트에 포함하여 LLM의 추출 정확도를 높임
- **품질 검증 시**: 추출 결과가 키워드 사전과 일치하는지 확인

In [ ]:
# Meta-Dictionary 정의
# 관계 타입별로 한국어 키워드를 매핑합니다.

meta_dictionary = {
    "FOUNDED_BY": {
        "keywords": ["설립", "창업", "세운", "만든", "대표"],
        "description": "기업 설립/창업 관계"
    },
    "DEVELOPS": {
        "keywords": ["개발", "출시", "만들다", "공개", "발표", "선보이다", "연구", "R&D"],
        "description": "제품/기술 개발 관계"
    },
    "INVESTS_IN": {
        "keywords": ["투자", "출자", "자금 투입", "펀딩", "지분", "매입", "보유", "인수"],
        "description": "투자/지분 관계"
    },
    "ACQUIRES": {
        "keywords": ["인수", "합병", "M&A", "매각", "흡수", "사들이다"],
        "description": "기업 인수/합병 관계"
    },
    "PARTNERS_WITH": {
        "keywords": ["협력", "제휴", "파트너십", "MOU", "합작", "공동 개발", "협약", "공급", "납품"],
        "description": "전략적 파트너십/협력 관계"
    },
    "COMPETES_WITH": {
        "keywords": ["경쟁", "대항", "맞서다", "라이벌", "vs", "시장 다툼", "점유율 경쟁"],
        "description": "경쟁 관계"
    },
    "EMPLOYS": {
        "keywords": ["CEO", "회장", "대표", "사장", "부회장", "임원", "소속", "근무"],
        "description": "고용/소속 관계"
    },
    "USES_TECH": {
        "keywords": ["사용", "활용", "적용", "탑재", "채택", "도입", "공정", "기반"],
        "description": "기술 활용 관계"
    },
    "PRODUCES": {
        "keywords": ["생산", "양산", "제조", "출하", "공장", "라인", "팹"],
        "description": "제품 생산 관계"
    }
}

# 저장
with open("data/meta_dictionary.json", "w", encoding="utf-8") as f:
    json.dump(meta_dictionary, f, ensure_ascii=False, indent=2)

print("=== Meta-Dictionary ===")
for rtype, info in meta_dictionary.items():
    kw_str = ", ".join(info["keywords"][:5])
    if len(info["keywords"]) > 5:
        kw_str += f" (+{len(info['keywords'])-5}개)"
    print(f"\n  [{rtype}]")
    print(f"    {info['description']}")
    print(f"    키워드: {kw_str}")

print(f"\n[OK] Meta-Dictionary를 data/meta_dictionary.json에 저장했습니다.")

In [ ]:
# Meta-Dictionary 활용 예제: 키워드로 관계 타입 추론

def find_relation_type(text):
    """텍스트에서 키워드를 찾아 가능한 관계 타입을 추론합니다."""
    matches = []
    for rtype, info in meta_dictionary.items():
        matched_keywords = [kw for kw in info["keywords"] if kw in text]
        if matched_keywords:
            matches.append({
                "relation_type": rtype,
                "matched_keywords": matched_keywords,
                "score": len(matched_keywords)
            })
    return sorted(matches, key=lambda x: x["score"], reverse=True)

# 테스트
test_sentences = [
    "국민연금이 삼성전자 지분을 추가 매입했다.",
    "SK하이닉스가 HBM4 양산을 시작했다.",
    "카카오와 삼성전자가 전략적 파트너십을 맺었다.",
    "삼성전자와 SK하이닉스의 HBM 시장 경쟁이 치열하다."
]

print("=== Meta-Dictionary 키워드 매칭 테스트 ===")
for sent in test_sentences:
    print(f"\n문장: \"{sent}\"")
    matches = find_relation_type(sent)
    if matches:
        for m in matches[:2]:  # 상위 2개만
            print(f"  -> {m['relation_type']} (매칭 키워드: {m['matched_keywords']})")
    else:
        print("  -> 매칭된 관계 타입 없음")

---
## 5. Neo4j 적재

추출된 엔티티와 관계를 Neo4j에 적재합니다.

### 핵심 포인트: MERGE vs CREATE

| 명령 | 동작 | 사용 시점 |
|------|------|----------|
| `CREATE` | 항상 새로 생성 (중복 허용) | 처음 한 번만 생성할 때 |
| `MERGE` | 있으면 조회, 없으면 생성 (중복 방지) | **대부분의 경우 추천** |

> **주의**: 여러 기사에서 같은 엔티티(예: 삼성전자)가 반복 등장합니다.  
> `CREATE`를 사용하면 "삼성전자" 노드가 여러 개 생기므로, 반드시 `MERGE`를 사용하세요!

In [ ]:
# Part 2 실습을 위해 데이터베이스 초기화
run_query("MATCH (n) DETACH DELETE n", print_result=False)
print("[OK] 데이터베이스 초기화 완료")

In [ ]:
# === 엔티티 적재 함수 ===

def load_entity(entity):
    """엔티티를 Neo4j에 MERGE로 적재합니다."""
    etype = entity["type"]
    name = entity["name"]
    props = entity.get("properties", {})
    
    # MERGE로 중복 방지
    # 동적 레이블은 Cypher에서 직접 지원하지 않으므로 타입별 분기
    query = f"""
        MERGE (n:{etype} {{name: $name}})
        ON CREATE SET n += $props
        ON MATCH SET n += $props
        RETURN n.name AS name, labels(n)[0] AS label
    """
    return run_query(query, {"name": name, "props": props}, print_result=False)

# === 관계 적재 함수 ===

def load_relation(relation):
    """관계를 Neo4j에 MERGE로 적재합니다."""
    source = relation["source"]
    target = relation["target"]
    rtype = relation["type"]
    props = relation.get("properties", {})
    
    # 관계 타입은 동적으로 설정할 수 없어서 APOC 없이는 분기 필요
    # 여기서는 간단하게 각 관계 타입별 쿼리를 생성
    query = f"""
        MATCH (a {{name: $source}}), (b {{name: $target}})
        MERGE (a)-[r:{rtype}]->(b)
        ON CREATE SET r += $props
        ON MATCH SET r += $props
        RETURN a.name AS from, type(r) AS rel, b.name AS to
    """
    return run_query(query, {"source": source, "target": target, "props": props}, print_result=False)

print("[OK] 적재 함수 준비 완료")

In [ ]:
# === 배치 적재 실행 ===

print("=== 추출 결과 Neo4j 적재 시작 ===")

entity_count = 0
relation_count = 0

for extraction in all_extractions:
    article_id = extraction["article_id"]
    print(f"\n--- {article_id} 적재 중 ---")
    
    # 1단계: 엔티티 적재
    for entity in extraction["entities"]:
        result = load_entity(entity)
        if result:
            entity_count += 1
            print(f"  [Entity] {entity['type']:12s} | {entity['name']}")
    
    # 2단계: 관계 적재
    for relation in extraction["relations"]:
        result = load_relation(relation)
        if result:
            relation_count += 1
            print(f"  [Rel]    {relation['source']} -[:{relation['type']}]-> {relation['target']}")

print(f"\n=== 적재 완료 ===")
print(f"  엔티티: {entity_count}개 처리")
print(f"  관계: {relation_count}개 처리")

---
## 6. 시각화 + 검증 쿼리

적재된 그래프가 올바른지 확인합니다.

### 검증 체크리스트

- [ ] 노드 수가 예상과 일치하는가?
- [ ] 관계 수가 예상과 일치하는가?
- [ ] 중복 노드가 없는가?
- [ ] 기대했던 관계가 존재하는가?

In [ ]:
# === 전체 그래프 확인 ===
print("=== 전체 그래프 통계 ===")

print("\n[1] 전체 노드 수:")
run_query("MATCH (n) RETURN count(n) AS 전체_노드_수")

print("\n[2] 전체 관계 수:")
run_query("MATCH ()-[r]->() RETURN count(r) AS 전체_관계_수")

print("\n[3] 레이블별 노드 수:")
run_query("""
    MATCH (n)
    RETURN labels(n)[0] AS 레이블, count(n) AS 노드수
    ORDER BY 노드수 DESC
""")

print("\n[4] 관계 타입별 수:")
run_query("""
    MATCH ()-[r]->() 
    RETURN type(r) AS 관계타입, count(r) AS 관계수
    ORDER BY 관계수 DESC
""")

In [ ]:
# === 모든 노드 목록 ===
print("=== 전체 노드 목록 ===")
run_query("""
    MATCH (n)
    RETURN labels(n)[0] AS 타입, n.name AS 이름
    ORDER BY 타입, 이름
""")

In [ ]:
# === 모든 관계 목록 ===
print("=== 전체 관계 목록 ===")
run_query("""
    MATCH (a)-[r]->(b)
    RETURN a.name AS 출발, type(r) AS 관계, b.name AS 도착
    ORDER BY 관계, 출발
""")

In [ ]:
# === 특정 패턴 검색 ===
print("=== 검증 쿼리 ===")

# 검증 1: 국민연금이 투자한 모든 기업
print("\n[검증 1] 국민연금이 투자한 기업:")
run_query("""
    MATCH (inv:Investment {name: '국민연금'})-[r:INVESTS_IN]->(c:Company)
    RETURN c.name AS 기업, r.amount AS 투자금액
""")

# 검증 2: 삼성전자가 개발하는 제품
print("\n[검증 2] 삼성전자가 개발하는 제품:")
run_query("""
    MATCH (c:Company {name: '삼성전자'})-[:DEVELOPS]->(p:Product)
    RETURN p.name AS 제품, p.category AS 카테고리
""")

# 검증 3: 2-hop - 국민연금이 투자한 기업이 개발하는 제품
print("\n[검증 3] 2-hop: 국민연금이 투자한 기업이 개발하는 제품:")
run_query("""
    MATCH (inv:Investment {name: '국민연금'})-[:INVESTS_IN]->(c:Company)-[:DEVELOPS]->(p:Product)
    RETURN inv.name AS 투자자, c.name AS 기업, p.name AS 제품
""")

# 검증 4: 3-hop - 국민연금이 투자한 기업이 사용하는 기술을 활용하는 다른 기업
print("\n[검증 4] 3-hop: 국민연금 투자 기업의 기술을 사용하는 다른 제품:")
run_query("""
    MATCH (inv:Investment {name: '국민연금'})
          -[:INVESTS_IN]->(c:Company)
          -[:USES_TECH]->(t:Technology)
          <-[:USES_TECH]-(c2:Company)
    WHERE c <> c2
    RETURN inv.name AS 투자자, c.name AS 투자기업, t.name AS 공통기술, c2.name AS 관련기업
""")

In [ ]:
# === 중복 검사 ===
print("=== 중복 노드 검사 ===")
print("\n같은 이름의 노드가 여러 개 있는지 확인:")
run_query("""
    MATCH (n)
    WITH n.name AS name, count(n) AS cnt
    WHERE cnt > 1
    RETURN name AS 중복이름, cnt AS 개수
""")

print("\n-> MERGE를 사용했으므로 중복이 없어야 합니다!")

### Neo4j Browser에서 시각화하기

Neo4j Browser (`http://localhost:7474`)에 접속하여 아래 쿼리를 실행하면  
그래프를 시각적으로 확인할 수 있습니다.

```cypher
// 전체 그래프 시각화
MATCH (n)-[r]->(m) RETURN n, r, m

// 삼성전자 중심 2-hop
MATCH path = (n)-[*1..2]-(samsung:Company {name: '삼성전자'})
RETURN path
```

---
## 7. 연습 문제

### 문제 1: 나머지 기사 7개에서 엔티티 추출

기사 4~10번에서 엔티티와 관계를 추출하세요.  
각 기사마다 최소 3개 엔티티, 2개 관계를 추출하는 것을 목표로 하세요.

> **팁**: 기사 4번(네이버-TSMC)부터 시작해보세요. 새로운 Company와 Technology가 등장합니다.

In [ ]:
# 기사 4번 확인
print(f"=== 기사 #{articles[3]['id']} ===")
print(f"제목: {articles[3]['title']}")
print(f"본문: {articles[3]['content']}")
print("\n힌트: 네이버(Company), TSMC(Company), 최수연(Person),")
print("      HyperCLOVA X(Product), 3나노 공정(Technology),")
print("      네이버클라우드(Company) 등이 있습니다.")

In [ ]:
# [연습 1] 기사 4번 추출 - 여기에 작성하세요

# extraction_04 = {
#     "article_id": "article_04",
#     "entities": [
#         {"name": "네이버", "type": "Company", "properties": {"industry": "IT"}},
#         # 더 추가하세요...
#     ],
#     "relations": [
#         {"source": "네이버", "target": "TSMC", "type": "PARTNERS_WITH", "properties": {}},
#         # 더 추가하세요...
#     ]
# }

# # 적재
# for entity in extraction_04["entities"]:
#     load_entity(entity)
# for relation in extraction_04["relations"]:
#     load_relation(relation)
# print("[OK] 기사 4번 적재 완료")

In [ ]:
# [연습 1 계속] 기사 5~10번도 같은 방식으로 추출하세요

# 기사 목록 다시 확인
for i in range(4, 10):
    print(f"\n[기사 {i+1}] {articles[i]['title']}")
    print(f"  {articles[i]['content'][:80]}...")

In [ ]:
# [연습 1] 기사 5~10번 추출 공간
# 각 기사에 대해 extraction_05 ~ extraction_10을 만들고 적재하세요.

# extraction_05 = { ... }
# extraction_06 = { ... }
# ...

print("기사 5~10번 추출을 완료한 후, 아래 셀에서 전체 그래프를 확인하세요.")

### 문제 2: Meta-Dictionary에 키워드 추가

기사 4~10번을 추출하면서 새로운 키워드를 발견했을 것입니다.  
Meta-Dictionary에 키워드를 추가하세요.

예시:
- `PARTNERS_WITH`: "MOU", "전략적 제휴", "공동 개발"
- `COMPETES_WITH`: "시장 다툼", "고객 유치 경쟁"
- `PRODUCES`: "팹 건설", "양산 라인"

In [ ]:
# [연습 2] Meta-Dictionary 키워드 추가

# 예시: 기사에서 발견한 새로운 키워드 추가
# meta_dictionary["PARTNERS_WITH"]["keywords"].extend(["공동 개발", "전략적 제휴"])
# meta_dictionary["COMPETES_WITH"]["keywords"].extend(["고객 유치", "수주 경쟁"])
# meta_dictionary["PRODUCES"]["keywords"].extend(["팹", "라인 가동"])

# # 업데이트된 사전 저장
# with open("data/meta_dictionary.json", "w", encoding="utf-8") as f:
#     json.dump(meta_dictionary, f, ensure_ascii=False, indent=2)
# print("[OK] Meta-Dictionary 업데이트 완료")

print("Meta-Dictionary에 새 키워드를 추가해보세요.")
print("Part 3에서 LLM 프롬프트에 이 사전을 활용합니다!")

---
## 정리

### 오늘 배운 것

| 항목 | 내용 |
|------|------|
| 온톨로지 설계 | 5개 엔티티 타입, 9개 관계 타입 정의 |
| 수동 엔티티 추출 | 뉴스 기사에서 엔티티/관계를 직접 추출 |
| Meta-Dictionary | 관계 타입별 키워드 매핑 사전 생성 |
| MERGE vs CREATE | 중복 방지를 위한 MERGE 사용 |
| 배치 적재 | Python 함수로 추출 결과 자동 적재 |
| 검증 쿼리 | 적재 결과의 정확성 확인 |

### 수작업의 한계

기사 3개를 추출하는 데 약 30분이 걸렸습니다.  
기사 1,000개라면? 기사 100,000개라면?

> **이것이 Part 3에서 LLM 자동 추출이 필요한 이유입니다!**

### 하지만 수작업이 중요한 이유

1. **Gold Standard**: 수동 추출 결과가 LLM 결과의 품질 기준이 됩니다.
2. **온톨로지 검증**: 설계한 스키마가 실제 데이터에 맞는지 확인할 수 있습니다.
3. **Meta-Dictionary**: 수작업 과정에서 발견한 키워드가 LLM 프롬프트를 개선합니다.

### 다음 Part 3 미리보기: LLM 기반 자동 추출

Part 3에서는:
- OpenAI GPT-4를 사용한 자동 엔티티/관계 추출
- 오늘 만든 온톨로지 + Meta-Dictionary를 프롬프트에 활용
- 수동 추출 결과와 자동 추출 결과 비교/평가

In [ ]:
# 세션 정리
# driver.close()
# print("[OK] Neo4j 드라이버 종료 완료")

print("Part 2 실습을 마칩니다.")
print("수고하셨습니다! Part 3에서 만나요!")